# MSigDB C2 Reactome Summary Wrangling
---


### Process used to collect data from SQLite DB

1. Go to https://www.gsea-msigdb.org/gsea/downloads.jsp
2. Scroll down to the MSigDB Section
3. Find "Human MSigDB SQLite database (ZIPped)"
    - The Human MSigDB v2023.2.Hs contents and metadata in the form of a (ZIPped) SQLite database. See our [documentation](https://docs.gsea-msigdb.org/#MSigDB/MSigDB_SQLite_Database/) for more details on the contents and usage. 
4. Download the database

![](https://docs.gsea-msigdb.org/MSigDB/SQLiteDBImages/msigdb_release.png)

**Note:** FOUNDER_NAMES, REFINEMENT_DATASETS, and VALIDATION_DATASETS are not available from the SQLiteDB, but are still accessible through the GSEA Website.

In [1]:
import sqlite3
con = sqlite3.connect("msigdb_v2023.2.Hs.db")

In [26]:
import pandas as pd

df = pd.read_sql_query(    """
    SELECT gene_set.standard_name,
        gene_set_details.description_brief,
        gene_set_details.description_full
    FROM gene_set
    INNER JOIN gene_set_details ON gene_set.id = gene_set_details.gene_set_id
    WHERE gene_set.standard_name = 'BERTUCCI_INVASIVE_CARCINOMA_DUCTAL_VS_LOBULAR_DN'
    """, con)

df

,standard_name,description_brief,description_full
0,BERTUCCI_INVASIVE_CARCINOMA_DUCTAL_VS_LOBULAR_DN,Genes down-regulated in the invasive ductal ca...,Invasive ductal carcinomas (IDCs) and invasive...


In [47]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

df = pd.read_sql_query("""
    SELECT gene_set.standard_name,
        gene_set_details.description_brief,
        gene_set_details.external_details_URL
    FROM gene_set
    INNER JOIN gene_set_details ON gene_set.id = gene_set_details.gene_set_id
    WHERE gene_set.collection_name = 'C2:CP:REACTOME'
    """, con)

df

,standard_name,description_brief,external_details_URL
0,REACTOME_INTERLEUKIN_6_SIGNALING,Interleukin-6 signaling,https://www.reactome.org/content/detail/R-HSA-...
1,REACTOME_APOPTOSIS,Apoptosis,https://www.reactome.org/content/detail/R-HSA-...
2,REACTOME_HEMOSTASIS,Hemostasis,https://www.reactome.org/content/detail/R-HSA-...
3,REACTOME_INTRINSIC_PATHWAY_FOR_APOPTOSIS,Intrinsic Pathway for Apoptosis,https://www.reactome.org/content/detail/R-HSA-...
4,REACTOME_MAPK3_ERK1_ACTIVATION,MAPK3 (ERK1) activation,https://www.reactome.org/content/detail/R-HSA-...
...,...,...,...
1687,REACTOME_SIGNALING_BY_THE_B_CELL_RECEPTOR_BCR,Signaling by the B Cell Receptor (BCR),https://www.reactome.org/content/detail/R-HSA-...
1688,REACTOME_FASTK_FAMILY_PROTEINS_REGULATE_PROCES...,FASTK family proteins regulate processing and ...,https://www.reactome.org/content/detail/R-HSA-...
1689,REACTOME_ION_CHANNEL_TRANSPORT,Ion channel transport,https://www.reactome.org/content/detail/R-HSA-...
1690,REACTOME_GLYCOSPHINGOLIPID_BIOSYNTHESIS,Glycosphingolipid biosynthesis,https://www.reactome.org/content/detail/R-HSA-...


In [45]:
df['external_details_URL']

0       https://www.reactome.org/content/detail/R-HSA-...
1       https://www.reactome.org/content/detail/R-HSA-...
2       https://www.reactome.org/content/detail/R-HSA-...
3       https://www.reactome.org/content/detail/R-HSA-...
4       https://www.reactome.org/content/detail/R-HSA-...
                              ...                        
1687    https://www.reactome.org/content/detail/R-HSA-...
1688    https://www.reactome.org/content/detail/R-HSA-...
1689    https://www.reactome.org/content/detail/R-HSA-...
1690    https://www.reactome.org/content/detail/R-HSA-...
1691    https://www.reactome.org/content/detail/R-HSA-...
Name: external_details_URL, Length: 1692, dtype: object

In [48]:
from bs4 import BeautifulSoup
import requests
import re

def get_summary_from_reactome_url(URL):
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find(class_="details-summation")
    results_no_html = re.sub('<[^<]+?>', '', str(results))
    results_clean = re.sub('\n', '', results_no_html)
    return results_clean

df['summary'] = df['external_details_URL'].progress_apply(get_summary_from_reactome_url)
df

100%|██████████| 1692/1692 [13:19<00:00,  2.12it/s] 


,standard_name,description_brief,external_details_URL,summary
0,REACTOME_INTERLEUKIN_6_SIGNALING,Interleukin-6 signaling,https://www.reactome.org/content/detail/R-HSA-...,Interleukin-6 (IL-6) is a pleiotropic cytokine...
1,REACTOME_APOPTOSIS,Apoptosis,https://www.reactome.org/content/detail/R-HSA-...,Apoptosis is a distinct form of cell death tha...
2,REACTOME_HEMOSTASIS,Hemostasis,https://www.reactome.org/content/detail/R-HSA-...,Hemostasis is a physiological response that cu...
3,REACTOME_INTRINSIC_PATHWAY_FOR_APOPTOSIS,Intrinsic Pathway for Apoptosis,https://www.reactome.org/content/detail/R-HSA-...,The intrinsic (Bcl-2 inhibitable or mitochondr...
4,REACTOME_MAPK3_ERK1_ACTIVATION,MAPK3 (ERK1) activation,https://www.reactome.org/content/detail/R-HSA-...,Mitogen-activated protein kinase kinase MAP2K1...
...,...,...,...,...
1687,REACTOME_SIGNALING_BY_THE_B_CELL_RECEPTOR_BCR,Signaling by the B Cell Receptor (BCR),https://www.reactome.org/content/detail/R-HSA-...,Mature B cells express IgM and IgD immunoglobu...
1688,REACTOME_FASTK_FAMILY_PROTEINS_REGULATE_PROCES...,FASTK family proteins regulate processing and ...,https://www.reactome.org/content/detail/R-HSA-...,Fas-activated serine/threonine kinase (FASTK) ...
1689,REACTOME_ION_CHANNEL_TRANSPORT,Ion channel transport,https://www.reactome.org/content/detail/R-HSA-...,Ion channels mediate the flow of ions across t...
1690,REACTOME_GLYCOSPHINGOLIPID_BIOSYNTHESIS,Glycosphingolipid biosynthesis,https://www.reactome.org/content/detail/R-HSA-...,The steps involved in the synthesis of glycosp...


In [50]:
df.isna().sum()

standard_name           0
description_brief       0
external_details_URL    0
summary                 0
dtype: int64

In [53]:
df.to_csv('msigdb_c2_reactome_summary.csv',index=False)